# Cython Tutorial
ref: https://www.youtube.com/watch?v=Ic1oE6SEOBs

In [6]:
!python -m pip install cython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.9 MB/s eta 0:00:00 MB/s eta 0:00:01


In [ ]:
def prime_finder_vanilla(count):
    primes = []
    
    found = 0
    number = 2
    while found < count:
        for x in primes:
            if number % x == 0:
                break
        else:
            primes.append(number)
            found += 1
        number += 1
        
    return primes


def prime_finder_optimized(int count):
    
    cdef int number, x, found
    cdef int primes[100000]  # arbitrarily large, fixed size array
    
    count = min(count, 100000)
    
    found = 0
    number = 2
    while found < count:
        for x in primes[:found]:
            if number % x == 0:
                break
        else:
            primes[found] = number
            found += 1
        number += 1
    
    return_list = [p for p in primes[:found]]
    return return_list

                
print(prime_finder_vanilla(10))

## Setup for Cython

1. In command shell, create cythonized source file, including C-extensions, and save with `.pyx` extension: `main.pyx`
1. In command shell, create setup.py
    ```python
    from setuptools import setup
    from Cython.Build import cythonize

    setup(
        ext_modules=cythonize('main.pyx')
    )
    
    ```
1. Build extension (or compile with C extension):
    ```
    python setup.py build_ext --inplace

    Compiling main.pyx because it changed.
    [1/1] Cythonizing main.pyx
    /mnt/beegfs/home/yroh/venv_py3913/lib/python3.9/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /mnt/beegfs/home/yroh/sandbox/cython/main.pyx
      tree = Parsing.p_module(s, pxd, full_module_name)
    running build_ext
    building 'main' extension
    creating build
    creating build/temp.linux-x86_64-3.9
    /usr/bin/gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -fPIC -fPIC -I/mnt/beegfs/home/yroh/venv_py3913/include -I/mnt/beegfs/software/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-9.4.0/python-3.9.13-3kr6zvqo2zlowhgyzj4ox6qfvbpgcglj/include/python3.9 -c main.c -o build/temp.linux-x86_64-3.9/main.o
    creating build/lib.linux-x86_64-3.9
    /usr/bin/gcc -pthread -shared -L/mnt/beegfs/software/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-9.4.0/bzip2-1.0.8-tncke62bbpty2zkapoo2y6imzc3mg7s7/lib -L/mnt/beegfs/software/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-9.4.0/expat-2.4.8-gsbkah7x4xek5oplbtotnztenoz4napi/lib -L/mnt/beegfs/software/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-9.4.0/gdbm-1.23-o7jzeqzshqsltt7zztlhyodthvwibihv/lib -L/mnt/beegfs/software/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-9.4.0/gettext-0.21.1-6xp3q5o5vprzxjmt24i7y72efmrogy4x/lib -L/mnt/beegfs/software/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-9.4.0/libffi-3.4.2-4js4tqr7aplvgugudrqivl2kdp5jluqe/lib -L/mnt/beegfs/software/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-9.4.0/ncurses-6.3-eupgjpk5754pj6uvss4agusnugnmvacb/lib -L/mnt/beegfs/software/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-9.4.0/openssl-1.1.1s-pgmns2wx4p2vmkwlzsl242xneqad7lwi/lib -L/mnt/beegfs/software/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-9.4.0/readline-8.1.2-ahpg44ypihqievdfqhbkaf2ml4dwl2nh/lib -L/mnt/beegfs/software/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-9.4.0/sqlite-3.39.4-taglkp3pwas6hvk4xmbz7wytnuby5ax4/lib -L/mnt/beegfs/software/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-9.4.0/util-linux-uuid-2.38.1-ogcpxdo4amufb56rd3ibsmakrwxnl4gl/lib -L/mnt/beegfs/software/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-9.4.0/xz-5.2.7-rflyw4ihch6tqqbmmot6wrltviyqqdlz/lib -L/mnt/beegfs/software/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-9.4.0/zlib-1.2.13-r7prbd7viq4mp4pi3bvvlscbxluhpljp/lib build/temp.linux-x86_64-3.9/main.o -L/mnt/beegfs/software/spack/opt/spack/linux-ubuntu20.04-zen2/gcc-9.4.0/python-3.9.13-3kr6zvqo2zlowhgyzj4ox6qfvbpgcglj/lib -o build/lib.linux-x86_64-3.9/main.cpython-39-x86_64-linux-gnu.so
    copying build/lib.linux-x86_64-3.9/main.cpython-39-x86_64-linux-gnu.so ->
    
    ```
1. Create `test.py` to call the defined functions and measure performance
    ```python
    import main
    import time

    start_vanilla = time.time()
    main.prime_finder_vanilla(50000)
    end_vanilla = time.time()

    print(end_vanilla - start_vanilla)

    start_c = time.time()
    main.prime_finder_optimized(50000)
    end_c = time.time()

    print(end_c - start_c)
    
    ```


In [8]:
!python test.py

6.478883266448975
0.8648514747619629


7 - 8X speed up!
<br>
<br>
**NOTE** It's big speed up, but according to the below lecture, Cython usually gives 100x even 1000x speed-up compared to regular Python. Need to investigate deeper on the above case.


-------
## Comparing codes: python, C/C++, cython
- Ref: https://www.youtube.com/watch?v=gMvkiQ-gOW8

### Python
```python
def fib(n):
    a, b = 1, 1
    for i in range(n):
        a, b = a + b, a
    return a
```
### C/C++
```c
int fib(int n)
{
    int tmp, i, a, b;
    a = b = 1;
    for (i = 0; i < n; i++) {
        tmp = a; a += b; b = tmp;
    }
    return a;
}
```
### Cython
```python
def fib(int n):
    cdef int i, a, b
    a, b = 1, 1
    for i in range(n):
        a, b = a + b, a
    return a
```

Cython just adds **static typing** of variables in Python code. Minimal changes. You can utilize existing Python code base, your knowledge of Python, and with little addition you get big performance boost.

### Hand-written extension module
```c
#include "Python.h"

static PyObject* fib(PyObject *self, PyObject *args)
{
    int n, a, b, i, tmp;
    if (!PyArg_ParseTuple(args, "i", &n))
        return NULL;
    a = b = 1;
    for (i=0; i<n; i++) {
        tmp = a; a += b; b = tmp;
    }
    return PyBuildValue("i", a);
}

static PyMethodDef ExampleMethods[] = {
    {"fib", fib, METH_VARAGS, ""},
    {NULL, NULL, 0, NULL}    /* Sentinel */
};

PyMODINIT_FUNC
initfib(void)
{
    (void) Py_InitModule("fib", ExampleMethods);
}
```

Compare this with Cython function. Also, often this hand-written extension modules are not faster than Cython codes, which does a lot of work underneath for efficient execution.<br>
Here are additional Cython features:
- Built-in support for NumPy
- Integrates well with IPython
- Foundational to Scientific Python ecosystem


## Cython Worklow
![Cython workflow](cython_flow.png)

## Comparison: CPython vs Cython
- Ref: https://blog.paperspace.com/boosting-python-scripts-cython/
### CPython both compiles and interprets
Python source (.py) -> CPython compiler -> CPython Bytecode (.pyc) -> CPython interpreter -> CPython VM -> Runs on different platform
